In [1]:
# This notebook cleans the data and creates a new csv file with the cleaned data

#### Library

In [2]:
import pandas as pd
# Set pandas options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

#### Data Loading

In [3]:
df = pd.read_csv('/home/kevin/Downloads/BESS/data/raw/2019/merged_df_2019.csv')

#### Data Cleaning

In [4]:
df.shape

(8763, 69)

In [5]:
df.head()

,datetime_,alberta_internal_load,forecast_alberta_internal_load,pool_price,forecast_pool_price,rolling_30day_avg_price,solar_generation,temp_calgary,temp_edmonton,temp_fortmc,wind_generation,ws_calgary,ws_edmonton,ws_fortmc,hour_of_day,day_of_week,day_of_month,week_of_month,month,year,is_winter,pool_price_lag_1,pool_price_lag_2,pool_price_lag_3,pool_price_lag_4,pool_price_lag_5,pool_price_lag_6,pool_price_lag_7,pool_price_lag_8,pool_price_lag_9,pool_price_lag_10,pool_price_lag_11,pool_price_lag_12,pool_price_lag_13,pool_price_lag_14,pool_price_lag_15,pool_price_lag_16,pool_price_lag_17,pool_price_lag_18,pool_price_lag_19,pool_price_lag_20,pool_price_lag_21,pool_price_lag_22,pool_price_lag_23,pool_price_lag_24,alberta_internal_load_lag_1,alberta_internal_load_lag_2,alberta_internal_load_lag_3,alberta_internal_load_lag_4,alberta_internal_load_lag_5,alberta_internal_load_lag_6,alberta_internal_load_lag_7,alberta_internal_load_lag_8,alberta_internal_load_lag_9,alberta_internal_load_lag_10,alberta_internal_load_lag_11,alberta_internal_load_lag_12,alberta_internal_load_lag_13,alberta_internal_load_lag_14,alberta_internal_load_lag_15,alberta_internal_load_lag_16,alberta_internal_load_lag_17,alberta_internal_load_lag_18,alberta_internal_load_lag_19,alberta_internal_load_lag_20,alberta_internal_load_lag_21,alberta_internal_load_lag_22,alberta_internal_load_lag_23,alberta_internal_load_lag_24
0,2019-01-01 00:00:00,10008.0,9753.0,32.90,33.14,43.16,0.0,-11.1,-10.1,-18.1,1126.509756,7.0,NaN,11.0,0,1,1,1,1,2019,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01 01:00:00,9868.0,9604.0,32.78,32.79,43.13,0.0,-8.7,-8.9,-17.0,1148.336475,7.0,NaN,9.0,1,1,1,1,1,2019,1,32.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01 02:00:00,9736.0,9473.0,32.78,32.79,43.10,0.0,-7.4,-7.5,-15.2,1091.469960,6.0,NaN,8.0,2,1,1,1,1,2019,1,32.78,32.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9868.0,10008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01 03:00:00,9597.0,9401.0,32.78,32.79,43.07,0.0,-4.9,-5.7,-13.2,1122.613398,8.0,NaN,12.0,3,1,1,1,1,2019,1,32.78,32.78,32.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9736.0,9868.0,10008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01 04:00:00,9530.0,9402.0,32.13,32.78,43.04,0.0,-0.9,-3.5,-11.5,1101.101023,4.0,NaN,9.0,4,1,1,1,1,2019,1,32.78,32.78,32.78,32.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9597.0,9736.0,9868.0,10008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.isnull().sum().sort_values(ascending=False)

ws_edmonton                       8763
pool_price_lag_24                   24
alberta_internal_load_lag_24        24
alberta_internal_load_lag_23        23
pool_price_lag_23                   23
pool_price_lag_22                   22
alberta_internal_load_lag_22        22
alberta_internal_load_lag_21        21
pool_price_lag_21                   21
pool_price_lag_20                   20
temp_edmonton                       20
alberta_internal_load_lag_20        20
pool_price_lag_19                   19
alberta_internal_load_lag_19        19
pool_price_lag_18                   18
alberta_internal_load_lag_18        18
pool_price_lag_17                   17
alberta_internal_load_lag_17        17
pool_price_lag_16                   16
alberta_internal_load_lag_16        16
alberta_internal_load_lag_15        15
pool_price_lag_15                   15
pool_price_lag_14                   14
alberta_internal_load_lag_14        14
alberta_internal_load_lag_13        13
pool_price_lag_13        

In [7]:
# Filter columns that do not start with 'alberta' or 'pool'
filtered_columns = [col for col in df.columns if not col.startswith('alberta_internal_load_') and not col.startswith('pool_price_')]

# Select columns with more than 1 missing value
columns_with_missing_values = df[filtered_columns].isnull().sum()
columns_with_missing_values = columns_with_missing_values[columns_with_missing_values > 0]

# Display the columns with more than 1 missing value
print(columns_with_missing_values.index)

Index(['alberta_internal_load', 'forecast_alberta_internal_load', 'pool_price',
       'forecast_pool_price', 'rolling_30day_avg_price', 'solar_generation',
       'temp_calgary', 'temp_edmonton', 'temp_fortmc', 'wind_generation',
       'ws_calgary', 'ws_edmonton', 'ws_fortmc'],
      dtype='object')


In [8]:
# impute median for missing values

for col in columns_with_missing_values.index:
    df[col].fillna(df[col].median(), inplace=True)

/home/kevin/Downloads/BESS/bess_venv/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [9]:
df.isnull().sum()

datetime_                            0
alberta_internal_load                0
forecast_alberta_internal_load       0
pool_price                           0
forecast_pool_price                  0
rolling_30day_avg_price              0
solar_generation                     0
temp_calgary                         0
temp_edmonton                        0
temp_fortmc                          0
wind_generation                      0
ws_calgary                           0
ws_edmonton                       8763
ws_fortmc                            0
hour_of_day                          0
day_of_week                          0
day_of_month                         0
week_of_month                        0
month                                0
year                                 0
is_winter                            0
pool_price_lag_1                     1
pool_price_lag_2                     2
pool_price_lag_3                     3
pool_price_lag_4                     4
pool_price_lag_5         

In [10]:
# fill NAN with bfill method
lagged_columns = [f'pool_price_lag_{i}' for i in range(1, 25)] + [f'alberta_internal_load_lag_{i}' for i in range(1, 25)]

df[lagged_columns] = df[lagged_columns].fillna(method='bfill')
# Verify if NaN values are filled
print(df[lagged_columns].isnull().sum().sort_values(ascending=False))

alberta_internal_load_lag_24    0
alberta_internal_load_lag_23    0
pool_price_lag_22               0
pool_price_lag_21               0
pool_price_lag_20               0
pool_price_lag_19               0
pool_price_lag_18               0
pool_price_lag_17               0
pool_price_lag_16               0
pool_price_lag_15               0
pool_price_lag_14               0
pool_price_lag_13               0
pool_price_lag_12               0
pool_price_lag_11               0
pool_price_lag_10               0
pool_price_lag_9                0
pool_price_lag_8                0
pool_price_lag_7                0
pool_price_lag_6                0
pool_price_lag_5                0
pool_price_lag_4                0
pool_price_lag_3                0
pool_price_lag_2                0
pool_price_lag_23               0
pool_price_lag_24               0
alberta_internal_load_lag_1     0
alberta_internal_load_lag_13    0
alberta_internal_load_lag_22    0
alberta_internal_load_lag_21    0
alberta_intern

In [11]:
df.isnull().sum().sort_values(ascending=False)

ws_edmonton                       8763
alberta_internal_load_lag_24         0
pool_price_lag_4                     0
year                                 0
is_winter                            0
pool_price_lag_1                     0
pool_price_lag_2                     0
pool_price_lag_3                     0
pool_price_lag_5                     0
week_of_month                        0
pool_price_lag_6                     0
pool_price_lag_7                     0
pool_price_lag_8                     0
pool_price_lag_9                     0
pool_price_lag_10                    0
pool_price_lag_11                    0
month                                0
day_of_month                         0
pool_price_lag_13                    0
day_of_week                          0
hour_of_day                          0
ws_fortmc                            0
ws_calgary                           0
wind_generation                      0
temp_fortmc                          0
temp_edmonton            

#### Save csv

In [12]:
df.to_csv('/home/kevin/Downloads/BESS/data/raw/2019/merged_df_2019_cleaned.csv', index=False)